In [6]:
import requests
import json
from tabulate import tabulate
# import coki.funcs as funcs
import funcs
import pandas as pd
from datetime import datetime
import os
import importlib

In [2]:
funcs.gen_table('GOGLD')

	 GOGLD - Cedear Google inc. - USD - 17:50:23
	 Last: 2.33 - High: 2.35 - Low: 2.2 - Volume: 15198
╭────────────────┬─────────────────┬────┬────────────────┬───────────────╮
│   Cant. compra │   Precio compra │    │   Precio venta │   Cant. venta │
├────────────────┼─────────────────┼────┼────────────────┼───────────────┤
│             50 │            2.20 │    │           2.33 │          4602 │
│              5 │            2.10 │    │           2.38 │           356 │
│             58 │            2.00 │    │           2.40 │           440 │
│             25 │            1.91 │    │           2.45 │           218 │
│                │                 │    │           2.46 │            50 │
╰────────────────┴─────────────────┴────┴────────────────┴───────────────╯


In [3]:
data_kod = funcs.get_ticker_data('KOD')
data_kod

{'short_ticker': 'KOD',
 'long_ticker': 'KOD-0003-C-CT-USD',
 'instrument_code': 'KO',
 'ext_code_cv': 8006,
 'instrument_name': 'Cedear Coca Cola Company',
 'instrument_short_name': 'Coca-Cola',
 'instrument_type': 'CEDEARS',
 'instrument_subtype': 'OTROS',
 'maturity': None,
 'expires_at': None,
 'logo_file_name': 'KO.jpg',
 'id_venue': 'BYMA',
 'id_session': 'CT',
 'id_segment': 'C',
 'settlement_days': 2,
 'currency': 'USD',
 'price_factor': 1,
 'contract_size': 1,
 'min_lot_size': 1,
 'id_security': 15468,
 'tick_size': 0.05,
 'date': '2023-12-18',
 'open': 11.7,
 'high': 11.9,
 'low': 11.6,
 'close': 11.7,
 'prev_close': 11.75,
 'last': 11.7,
 'bid': 11.3,
 'ask': 12.3,
 'bids': [{'size': 5, 'price': 11.3},
  {'size': 10, 'price': 10.8},
  {'size': 10, 'price': 9.1}],
 'asks': [{'size': 139, 'price': 12.3},
  {'size': 396, 'price': 12.4},
  {'size': 1, 'price': 12.5},
  {'size': 95, 'price': 13},
  {'size': 129, 'price': 13.4}],
 'turnover': 799361.75,
 'volume': 68464,
 'variati

In [ ]:
funcs.gen_table_orders(ticker=ticker, filtro=filtro)

In [10]:
def gen_table_orders(account_id, ticker="ALL", filtro="cancelado"):
    orders = funcs.get_orders_link(account_id)

    tables = []
    for i in range(0, len(orders)):
        data = orders[i]

        if data["currency"] == "USD":
            ticker_name = data["ticker"] + "D"
        else:
            ticker_name = data["ticker"]
        
        if data["status"] == 'MARKET':
            ejecutadas = 0
        else:
            ejecutadas = data["result_quantity"]
        table = {
            "Hora": datetime.strptime(data["date"], "%Y-%m-%dT%H:%M:%S.%f%z")
            .time()
            .strftime("%H:%M"),
            "Ticker": ticker_name,
            "Estado": data["status"],
            "Operación": data["order_type"],
            "Precio": round(data["set_price"], 2),
            "Cantidad": int(round(data["set_quantity"], 0)),
            "Monto": int(round(data["set_amount"], 2)),
            "Ejecutadas": ejecutadas,
            "code_op": data["order_id"],
        }
        tables.append(table)

    df = pd.DataFrame(tables)
    df = df.copy().sort_values(by=["Hora"], ascending=True).reset_index(drop=True)
    df["Op"] = df.index
    df = df.copy().set_index("Op")

    df["Estado"] = df["Estado"].replace(
        {
            "MARKET": "En mercado",
            "CANCELLED": "Cancelada",
            "REJECTED": "Rechazada",
            "EXECUTED": "Ejecutada",
            "PARTIALLY_EXECUTED": "Ejecutada parcialmente",
        }
    )

    if filtro == "cancelado":
        df = df[(df["Estado"] != "Cancelada") & (df["Estado"] != "Rechazada")].copy()
    elif filtro == "ejecutado":
        df = df[df["Estado"] == "Ejecutada"].copy()
    elif filtro == "mercado":
        df = df[
            (df["Estado"] == "Ejecutada parcialmente") | (df["Estado"] == "En mercado")
        ].copy()

    df["Operación"] = df["Operación"].replace(
        {
            "BUY": "Compra",
            "SELL": "Venta",
        }
    )

    if ticker != "ALL":
        df = df[df["Ticker"] == ticker].copy()

    df = df.sort_values(by=["Ticker", "Hora"], ascending=False)

    dict_operaciones = df["code_op"].to_dict()

    # print datos

    time_str = datetime.now().strftime("%H:%M:%S")

    print(
        "\t", 
        time_str,
        "-",
        account_id,
        "-",
        data["short_ticker"],
        "-",
        data["instrument_name"],
        "-",
        data["currency"],
        "-",
        ,
    )

    print(
        tabulate(
            df[df.columns[:-1]],
            headers="keys",
            tablefmt="rounded_outline",
            floatfmt=".2f",
            stralign="center",
        )
    )
    return dict_operaciones


In [11]:
gen_table_orders(55743)

╭──────┬────────┬──────────┬────────────┬─────────────┬──────────┬────────────┬─────────┬──────────────╮
│   Op │  Hora  │  Ticker  │   Estado   │  Operación  │   Precio │   Cantidad │   Monto │   Ejecutadas │
├──────┼────────┼──────────┼────────────┼─────────────┼──────────┼────────────┼─────────┼──────────────┤
│   15 │ 12:14  │  TSLAD   │ Ejecutada  │    Venta    │    16.90 │         10 │     169 │           10 │
│   12 │ 11:43  │  TSLAD   │ Ejecutada  │   Compra    │    16.65 │         10 │     166 │           10 │
│   28 │ 14:32  │  METAD   │ En mercado │   Compra    │    13.95 │         15 │     209 │            0 │
│   30 │ 15:08  │  MELID   │ En mercado │   Compra    │    26.40 │         15 │     396 │            0 │
│   27 │ 14:29  │  MELID   │ Ejecutada  │    Venta    │    26.50 │         10 │     265 │           10 │
│   17 │ 12:51  │  MELID   │ Ejecutada  │   Compra    │    26.40 │         10 │     264 │           10 │
│    8 │ 11:33  │  MELID   │ Ejecutada  │    Venta    │

{15: '8ff529c1-a326-4e3e-bbfa-54717fcc697a',
 12: 'e8b799e1-468d-4058-b041-d236d64d6b76',
 28: 'b31b99da-9d09-44c6-94bd-b55e0b7d7474',
 30: '8af15ef1-cf48-411a-9ec0-11e536ddb250',
 27: '9c411cb1-6e78-431c-96ed-1e6afe478185',
 17: '480bd668-e135-4551-a5f2-3972002d0141',
 8: '06174b92-2de1-4e86-bb1f-2c6968e88cb3',
 29: '341dd469-0711-4fcd-b24b-4055b6174930',
 23: '10459f62-860f-4d77-bd97-f1ee456997af',
 20: 'c7393fd0-2bea-4874-b657-45dc54141c8b',
 18: 'ff05a416-620b-430d-922c-6a20a1fb11eb',
 16: 'ad7ab74b-399f-4e2b-9256-e751a432e6ad',
 9: 'dc5b09c1-9e58-415d-84c7-21d33ff2275c',
 6: '0f9c935f-e520-42f7-bad0-5d6972c64a0d',
 5: 'b3310712-8ce7-4aa8-809d-55bb9e9899a1',
 4: '817b989a-446e-40c5-8f81-66f4662bccaa',
 2: 'bbf1de71-8701-4901-95a9-d26ff7490244',
 0: '8592965c-46ce-448a-bdcf-195292423cc3',
 26: '30b8eb6b-9aed-40d6-ad39-a2b9bdae549b',
 25: '7067a0eb-b941-41ee-93c8-9c56a34cf2a9',
 21: '62e31d36-4b7f-4387-8023-28211b0f028f',
 13: '64c97e3e-f8c3-4b83-9659-3523537a75a9',
 11: 'f970ef4c-61